# 필요한 라이브러리 및 환경 설정

In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import cv2
import numpy as np
from keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img
import glob
import pandas as pd 
from natsort import natsorted

In [2]:
image_size = (180, 180)
channels = 3
#본 모델에서 180, 180 3채널로 고정시켰기 때문에 고정변수입니다 이미지사이즈

In [3]:
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# 모델 불러오기, 분류 class 설정

In [4]:
exposure_model = load_model('/content/drive/MyDrive/ttproject/model/newVGG16(노출비노출).h5')
violence_model = load_model('/content/drive/MyDrive/ttproject/model/newVGG16(폭력비폭력).h5')
relationship_model = load_model('/content/drive/MyDrive/ttproject/model/newVGG16(관계비관계).h5')

In [5]:
naked_classes = ['Naked', 'Non-Naked'] 
violence_classes = ['Non-Violence', 'Violence'] 
sexual_classes = ['Sexual', 'Non-Sexual']     

In [6]:
def preprocess_image(image):
  image = cv2.resize(image, image_size)
  image = image.astype('float32') / 255.0
  return image

In [16]:
def classify_image(image):
    preprocessed_image = preprocess_image(image)
    naked_prediction = exposure_model.predict(np.expand_dims(preprocessed_image, axis=0))[0]
    violence_prediction = violence_model.predict(np.expand_dims(preprocessed_image, axis=0))[0]
    sexual_prediction = relationship_model.predict(np.expand_dims(preprocessed_image, axis=0))[0]

    # 확률이 0.85 이상일 경우 해당 클래스를 반환, 그렇지 않으면 'Uncertain' 반환
    naked_class = naked_classes[np.argmax(naked_prediction)] if np.max(naked_prediction) > 0.85 else 'Uncertain'
    violence_class = violence_classes[np.argmax(violence_prediction)] if np.max(violence_prediction) > 0.85 else 'Uncertain'
    sexual_class = sexual_classes[np.argmax(sexual_prediction)] if np.max(sexual_prediction) > 0.85 else 'Uncertain'

    return naked_class, violence_class, sexual_class

### 영상 개별 테스트

In [19]:
# 영상 디렉토리
path="/content/drive/MyDrive/ttproject/tiktok_video_카테고리별/건전영상/건전영상001"
filenames = natsorted(os.listdir(path))

frame_df = pd.DataFrame({"filename":filenames})
frame_df["path"] = frame_df["filename"].apply(lambda x: os.path.join(path, x))
frame_df[["노출", "폭력", "관계"]] = frame_df["path"].apply(lambda x: pd.Series(classify_image(cv2.imread(x))))
frame_df = frame_df.drop(columns=['path'])

frame_df

1/1 [==============================] - 0s 21ms/step


,filename,노출,폭력,관계
0,frame_0.jpg,Non-Naked,Non-Violence,Non-Sexual
1,frame_1.jpg,Non-Naked,Non-Violence,Non-Sexual
2,frame_2.jpg,Non-Naked,Non-Violence,Non-Sexual
3,frame_3.jpg,Non-Naked,Non-Violence,Non-Sexual
4,frame_4.jpg,Non-Naked,Non-Violence,Non-Sexual
5,frame_5.jpg,Non-Naked,Non-Violence,Non-Sexual
6,frame_6.jpg,Non-Naked,Non-Violence,Non-Sexual
7,frame_7.jpg,Non-Naked,Non-Violence,Non-Sexual
8,frame_8.jpg,Non-Naked,Non-Violence,Non-Sexual
9,frame_9.jpg,Non-Naked,Non-Violence,Non-Sexual


In [20]:
folder_df = pd.DataFrame({
    "폴더명": [os.path.basename(path)],
    "유해/비유해": ["유해" if any(frame_df["노출"] == 'Naked') or any(frame_df["폭력"] == 'Violence') or any(frame_df["관계"] == 'Sexual') else "비유해"],
    "노출 예측": ["Naked" if any(frame_df["노출"] == "Naked") else "Non-Naked"],
    "폭력 예측": ["Violence" if any(frame_df["폭력"] == "Violence") else "Non-Violence"],
    "관계 예측": ["Sexual" if any(frame_df["관계"] == "Sexual") else "Non-Sexual"]
})

folder_df

,폴더명,유해/비유해,노출 예측,폭력 예측,관계 예측
0,건전영상001,비유해,Non-Naked,Non-Violence,Non-Sexual


## 폴더 전체 분류 테스트

In [21]:
# 틱톡 전체 영상 root 디렉토리
root_path = "/content/drive/MyDrive/ttproject/tiktok_video"
folder_df = pd.DataFrame(columns=["폴더명", "유해/비유해", "노출 예측", "폭력 예측", "관계 예측"])

for root, dirs, files in os.walk(root_path):
    for folder in dirs:
        folder_path = os.path.join(root, folder)
        folder_name = os.path.basename(folder_path)
        filenames = natsorted(os.listdir(folder_path))

        frame_df = pd.DataFrame({"filename": filenames})
        frame_df["path"] = frame_df["filename"].apply(lambda x: os.path.join(folder_path, x))
        frame_df[["노출", "폭력", "관계"]] = frame_df["path"].apply(lambda x: pd.Series(classify_image(cv2.imread(x))))
        frame_df = frame_df.drop(columns=["path"])

        is_harmful = any(frame_df["노출"] == "Naked") or any(frame_df["폭력"] == "Violence") or any(frame_df["관계"] == "Sexual")
        folder_df = folder_df.append({
            "폴더명": folder_name,
            "유해/비유해": "유해" if is_harmful else "비유해",
            "노출 예측": "Naked" if any(frame_df["노출"] == "Naked") else "Non-Naked",
            "폭력 예측": "Violence" if any(frame_df["폭력"] == "Violence") else "Non-Violence",
            "관계 예측": "Sexual" if any(frame_df["관계"] == "Sexual") else "Non-Sexual"
        }, ignore_index=True)

folder_df = folder_df.sort_values("폴더명", ascending=True).reset_index(drop=True)
folder_df

1/1 [==============================] - 0s 19ms/step


<ipython-input-21-d10a1825f848>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folder_df = folder_df.append({


1/1 [==============================] - 0s 20ms/step


<ipython-input-21-d10a1825f848>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folder_df = folder_df.append({


1/1 [==============================] - 0s 35ms/step


...

1/1 [==============================] - 0s 19ms/step


<ipython-input-21-d10a1825f848>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folder_df = folder_df.append({


1/1 [==============================] - 0s 21ms/step


<ipython-input-21-d10a1825f848>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  folder_df = folder_df.append({


,폴더명,유해/비유해,노출 예측,폭력 예측,관계 예측
0,Naked001,유해,Naked,Non-Violence,Sexual
1,Naked002,유해,Naked,Violence,Sexual
2,Naked003,유해,Naked,Violence,Sexual
3,Naked004,유해,Naked,Non-Violence,Sexual
4,Naked005,유해,Naked,Non-Violence,Non-Sexual
...,...,...,...,...,...
95,Violence006,유해,Naked,Non-Violence,Sexual
96,Violence007,유해,Non-Naked,Violence,Sexual
97,Violence008,유해,Non-Naked,Violence,Sexual
98,Violence009,유해,Non-Naked,Violence,Sexual


In [ ]:
# 위에 결과 나오면 폴더명을 키값으로 아래 output_df에 붙여넣기 하면 끄읏

In [ ]:
output_df = pd.read_excel('/content/drive/MyDrive/ttproject/필터링결과표.xlsx')
output_df

,폴더명,유해/비유해 여부,노출 여부,폭력 여부,관계 여부,유해/비유해 예측,노출 예측,폭력 예측,관계 예측,blind 여부 (폴더 위치)
0,건전영상001,unharmful,Non-Naked,Non-Violence,Non-Sexual,NaN,NaN,NaN,NaN,NaN
1,건전영상002,unharmful,Non-Naked,Non-Violence,Non-Sexual,NaN,NaN,NaN,NaN,NaN
2,건전영상003,unharmful,Non-Naked,Non-Violence,Non-Sexual,NaN,NaN,NaN,NaN,NaN
3,건전영상004,unharmful,Non-Naked,Non-Violence,Non-Sexual,NaN,NaN,NaN,NaN,NaN
4,건전영상005,unharmful,Non-Naked,Non-Violence,Non-Sexual,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
95,폭력영상006,harmful,Non-Naked,Violence,Non-Sexual,NaN,NaN,NaN,NaN,NaN
96,폭력영상007,harmful,Non-Naked,Violence,Non-Sexual,NaN,NaN,NaN,NaN,NaN
97,폭력영상008,harmful,Non-Naked,Violence,Non-Sexual,NaN,NaN,NaN,NaN,NaN
98,폭력영상009,harmful,Non-Naked,Violence,Non-Sexual,NaN,NaN,NaN,NaN,NaN
